In [ ]:
import pandas as pd
import sys
sys.path.extend([
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava',
    r'C:\Users\Usuario\OneDrive\Desktop\Learning-Testing\PyStrava\Scripts'
])
from Functions import *
from StravaCredentials import *
import json
from sqlalchemy import create_engine
import psycopg2
import pandas.io.sql as sqlio

In [ ]:
db_url = f'postgresql://{username}:{pwd}@{hostname}:{port_id}/{database}'
engine = create_engine(db_url)

In [4]:
conn = psycopg2.connect(
    host = hostname,
    dbname = database,
    user = username,
    password = pwd,
    port = port_id
)

cur = conn.cursor()
query = "select activity_id from activity"

df = sqlio.read_sql_query(query, conn)
list(df['activity_id'])

In [ ]:
# Cleaning json and creating score columns
workout_df = CleanWorkoutJson(workout_json)
laps_df = CleanLapsJson(workout_json)
workout_df = CreateScoreColumns(workout_df)

In [ ]:
# Breaking down workout_df to all different tables - laps table is = to laps_df
def DivideTables(workout_df):    
    activity = workout_df[['activity_id',
                        'start_date',
                        'sport_type',
                        'effort_score_label']]

    activity_name = workout_df[['activity_id',
                                'name']]

    activity_coordinates = workout_df[['activity_id',
                                    'start_lat',
                                    'start_long',
                                    'end_lat',
                                    'end_long']]

    activity_details = workout_df[['activity_id',
                                'workout_time_min',
                                'calories',
                                'average_heartrate',
                                'avg_lap_time',
                                'total_elevation_gain',
                                'max_heartrate',
                                'distance',
                                'average_speed_km/h',
                                'max_speed_km/h',
                                'average_temp',
                                'pace',
                                'lap_count']]

    activity_scores = workout_df[['activity_id',
                                'distance_score',
                                'workout_time_score',
                                'calorie_score',
                                'total_elevation_gain_score',
                                'average_heartrate_score',
                                'max_heartrate_score',
                                'avg_lap_time_score',
                                'lap_count_score',
                                'avg_speed_score',
                                'max_speed_score',
                                'pace_score',
                                'effort_score',
                                'effort_score_rank']]
    
    return activity, activity_name, activity_coordinates, activity_details, activity_scores

In [ ]:
all_workouts_df = pd.read_csv(r'C:\Users\Usuario\Desktop\Learning-Testing\PyStrava\Outputs\all_workouts_df.csv')

In [ ]:
activity, activity_name, activity_coordinates, activity_details, activity_scores = DivideTables(all_workouts_df)

In [ ]:
# Sending data to Postgresql
activity.to_sql('activity', engine, if_exists='replace', index=False)
activity_name.to_sql('activity_name', engine, if_exists='replace', index=False)
activity_coordinates.to_sql('activity_coordinates', engine, if_exists='replace', index=False)
activity_details.to_sql('activity_details', engine, if_exists='replace', index=False)
activity_scores.to_sql('activity_scores', engine, if_exists='replace', index=False)
laps_df.to_sql('laps', engine, if_exists='replace', index=False)